# Attachment generator

In [ ]:
import os

def create_exact_size_file(file_path, size_mb):
    # Calculate the exact size in bytes
    size_bytes = size_mb * 1024 * 1024

    # Open the file in write binary ('wb') mode
    with open(file_path, 'wb') as file:
        file.seek(size_bytes - 1)  # Move to the position one byte before the desired size
        file.write(b'\0')  # Write a single zero byte at this position

def generate_files(directory_path, excel_filename, pdf_filename, target_size_mb):
    # Ensure the directory ends with a slash
    if not directory_path.endswith('/'):
        directory_path += '/'

    # Generate exact-sized Excel and PDF files
    create_exact_size_file(directory_path + excel_filename, target_size_mb)
    create_exact_size_file(directory_path + pdf_filename, target_size_mb)

    print(f'Generated Excel file at {directory_path + excel_filename} with size exactly {target_size_mb} MB')
    print(f'Generated PDF file at {directory_path + pdf_filename} with size exactly {target_size_mb} MB')

# Set the directory where files will be saved
directory_path = "/Users/ikawahyuni/dataquality/dummy_files"

# Set filenames and target size
generate_files(directory_path, 'dummy_15mb.xlsx', 'dummy_15mb.pdf', 15)


# 14 MB

In [1]:
import os

def create_exact_size_file(file_path, size_mb):
    # Calculate the exact size in bytes
    size_bytes = size_mb * 1024 * 1024

    # Open the file in write binary ('wb') mode
    with open(file_path, 'wb') as file:
        file.seek(size_bytes - 1)  # Move to the position one byte before the desired size
        file.write(b'\0')  # Write a single zero byte at this position

def generate_files(directory_path, excel_filename, pdf_filename, target_size_mb):
    # Ensure the directory ends with a slash
    if not directory_path.endswith('/'):
        directory_path += '/'

    # Generate exact-sized Excel and PDF files
    create_exact_size_file(directory_path + excel_filename, target_size_mb)
    create_exact_size_file(directory_path + pdf_filename, target_size_mb)

    print(f'Generated Excel file at {directory_path + excel_filename} with size exactly {target_size_mb} MB')
    print(f'Generated PDF file at {directory_path + pdf_filename} with size exactly {target_size_mb} MB')

# Set the directory where files will be saved
directory_path = "/Users/ikawahyuni/dataquality/dummy_files"

# Set filenames and target size (change from 15 to 14)
generate_files(directory_path, 'dummy_14mb.xlsx', 'dummy_14mb.pdf', 14)


Generated Excel file at /Users/ikawahyuni/dataquality/dummy_files/dummy_14mb.xlsx with size exactly 14 MB
Generated PDF file at /Users/ikawahyuni/dataquality/dummy_files/dummy_14mb.pdf with size exactly 14 MB


# maximum size limit of 10,485,760 bytes (10 MB)

In [3]:
import os

def create_exact_size_file(file_path, size_mb):
    # Maximum size in bytes (10 MB)
    max_size_bytes = 9485760

    # Calculate the desired size in bytes
    size_bytes = size_mb * 1024 * 1024

    # Adjust size if it exceeds the maximum allowed
    if size_bytes > max_size_bytes:
        size_bytes = max_size_bytes

    # Open the file in write binary ('wb') mode
    with open(file_path, 'wb') as file:
        file.seek(size_bytes - 1)  # Move to the position one byte before the desired size
        file.write(b'\0')  # Write a single zero byte at this position

def generate_files(directory_path, excel_filename, pdf_filename, target_size_mb):
    # Ensure the directory ends with a slash
    if not directory_path.endswith('/'):
        directory_path += '/'

    # Generate exact-sized Excel and PDF files
    create_exact_size_file(directory_path + excel_filename, target_size_mb)
    create_exact_size_file(directory_path + pdf_filename, target_size_mb)

    actual_size_mb = min(target_size_mb, 10)  # Limit the reported size to 10 MB max
    print(f'Generated Excel file at {directory_path + excel_filename} with size up to {actual_size_mb} MB')
    print(f'Generated PDF file at {directory_path + pdf_filename} with size up to {actual_size_mb} MB')

# Set the directory where files will be saved
directory_path = "/Users/ikawahyuni/dataquality/dummy_files"

# Set filenames and target size (e.g., 12 MB would be reduced to 10 MB)
generate_files(directory_path, 'dummy.xlsx', 'dummy.pdf', 12)


Generated Excel file at /Users/ikawahyuni/dataquality/dummy_files/dummy.xlsx with size up to 10 MB
Generated PDF file at /Users/ikawahyuni/dataquality/dummy_files/dummy.pdf with size up to 10 MB


In [15]:
import os
import zipfile
import rarfile  # pip install rarfile
import pandas as pd

def extract_files(directory):
    data = []  # List to store data for the DataFrame

    # Iterate over all the files in the given directory
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        try:
            if filename.endswith(".zip"):
                # If the file is a ZIP file, extract it
                with zipfile.ZipFile(filepath, 'r') as zip_ref:
                    extract_path = os.path.join(directory, filename[:-4])  # Creates a folder named after the file
                    zip_ref.extractall(extract_path)
                    print(f"Extracted {filename} to {extract_path}")
            elif filename.endswith(".rar"):
                # If the file is a RAR file, extract it
                with rarfile.RarFile(filepath, 'r') as rar_ref:
                    extract_path = os.path.join(directory, filename[:-4])  # Creates a folder named after the file
                    rar_ref.extractall(extract_path)
                    print(f"Extracted {filename} to {extract_path}")
            else:
                continue

            # After extraction, delete the archive file
            os.remove(filepath)
            print(f"Deleted {filename} after extraction.")

        except Exception as e:
            print(f"Failed to process {filename}: {e}")

    # Collect all file paths in the newly created directories
    for dirpath, dirnames, filenames in os.walk(directory):
        for file in filenames:
            if not file.endswith(('.zip', '.rar')):
                full_path = os.path.join(dirpath, file)
                data.append({"Title": file, 
                             "Description": "", 
                             "VersionData": full_path, 
                             "FirstPublishLocationId": ""})

    # Create DataFrame
    df = pd.DataFrame(data)
    return df

# Specify the directory to scan for ZIP and RAR files
directory_path = '/Users/ikawahyuni/Downloads/extractzip'
df = extract_files(directory_path)

# Optionally, save to CSV
df=df.iloc[:2]
df.to_csv(directory_path+"/extracted_files_info.csv", index=False)
df


,Title,Description,VersionData,FirstPublishLocationId
0,.DS_Store,,/Users/ikawahyuni/Downloads/extractzip/.DS_Store,
1,extracted_files_info.csv,,/Users/ikawahyuni/Downloads/extractzip/extract...,


500MR000003qcx7YAA